### Introduction: Business Problem

São Paulo is a huge city located in the State of São Paulo, Brazil. There are lots of companies locate there and every year and people from all over the world come to São Paulo for business trips. There are many Hotels in São Paulo, but many of them are not specific for business.

So, this project aims to find the best location for a Hotel specific for business trip. Specifically, it is targeted to stakeholders interested in opening a Hotel in São Paulo, Brazil. Using Data Science and Foursquare location data we will try to find a location with no hotels nearby.

### Data

The following factors will influence our decision:

number of existing hotels in the neighborhood

I will use the spaced grid of locations, centered around São Paulo, to define the best neighborhood.

Following data sources will be needed to extract/generate the required information:

Google Maps API will be used to find the coordinates of the neighborhoods.

Foursquare API will be used to get the number of companies, malls and supermarkets.

### Installing Libraries

In [1]:
!pip install folium

In [2]:
!pip install googlemaps

In [3]:
!pip install geocoder

In [4]:
!pip install google_api

In [5]:
!pip install geopy

In [6]:
!pip install geopandas

### Importing Libraries

In [7]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

from googlemaps import Client as GoogleMaps

import folium # map rendering library

### Reading the data set

In [8]:
neighborhoods=pd.read_csv('https://raw.githubusercontent.com/natvilanova/DSrepository/master/neighborhoods.csv')

In [9]:
neighborhoods

,Neighborhoods
0,Alto de Pinheiros
1,Anhanguera
2,Aricanduva
3,Artur Alvim
4,Barra Funda
5,Bela Vista
6,Belem
7,Bom Retiro
8,Bras
9,Brasilandia


### Getting Coordinates

In [10]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Sao Paulo, Brazil'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [11]:
coords = [ get_latlng(neighborhood) for neighborhood in neighborhoods["Neighborhoods"].tolist() ]

In [12]:
coords

[[-23.55272999999994, -46.70915999999994],
 [-23.420969999999954, -46.78516999999994],
 [-23.567709999999977, -46.510249999999985],
 [-23.551049999999975, -46.47999999999996],
 [-23.52909999999997, -46.662079999999946],
 [-21.599999999999966, -46.599999999999966],
 [-23.53804999999994, -46.59591999999998],
 [-23.524509999999964, -46.640709999999956],
 [-23.542769999999962, -46.61999999999995],
 [-23.469499999999982, -46.68954999999994],
 [-23.570889999999963, -46.709679999999935],
 [-22.733329999999967, -46.58332999999993],
 [-23.563369999999964, -46.61991999999998],
 [-23.62035999999995, -46.67229999999995],
 [-23.95632999999998, -46.337789999999984],
 [-23.63934999999998, -46.76667999999995],
 [-23.506019999999978, -46.528219999999976],
 [-23.66289999999998, -47.62089999999995],
 [-23.711789937244095, -46.70030002553517],
 [-23.54797999999994, -46.53884999999997],
 [-23.511519999999962, -46.656289999999956],
 [-23.670849999999973, -46.65762999999998],
 [-23.55842999999993, -46.477259

In [14]:
dfneigh = pd.DataFrame(coords, columns=['Lat', 'Lon'])

In [15]:
dfneigh.head()

,Lat,Lon
0,-23.55273,-46.70916
1,-23.42097,-46.78517
2,-23.56771,-46.51025
3,-23.55105,-46.48000
4,-23.52910,-46.66208


In [16]:
neighborhoods['Latitude'] = dfneigh['Lat']
neighborhoods['Longitude'] = dfneigh['Lon']

In [27]:
neighborhoods

,Neighborhoods,Latitude,Longitude
0,Alto de Pinheiros,-23.552730,-46.709160
1,Anhanguera,-23.420970,-46.785170
2,Aricanduva,-23.567710,-46.510250
3,Artur Alvim,-23.551050,-46.480000
4,Barra Funda,-23.529100,-46.662080
5,Bela Vista,-21.600000,-46.600000
6,Belem,-23.538050,-46.595920
7,Bom Retiro,-23.524510,-46.640710
8,Bras,-23.542770,-46.620000
9,Brasilandia,-23.469500,-46.689550


In [19]:
neighborhoods.shape

(97, 3)

In [20]:
address = 'Sao Paulo, Brasil'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Sao Paulo, Brasil {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Sao Paulo, Brasil -23.5506507, -46.6333824.


### Creating São Paulo Map with neghborhoods

In [21]:
neighborhoods_map = folium.Map(location=[latitude, longitude], zoom_start=8)

In [22]:
neighborhoods_map

In [25]:
for lat, lng, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhoods']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(neighborhoods_map)  

In [26]:
neighborhoods_map

In [66]:
neighborhoods_map.save('neighborhoods_map.html')

### Using Foursquare to analyse the neighborhoods

In [30]:
# define Foursquare Credentials and Version
CLIENT_ID = 'OCQYHN0P51EK43M4QCL22UAB1IYKFJVAGB0I4DQ4DKTNDVH1' # your Foursquare ID
CLIENT_SECRET = 'O5YLENJUJPGDQQYNSBB2MU3MSFATXSR2A3YEUCSNAPLGIMDW' # your Foursquare Secret
VERSION = '20200102' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OCQYHN0P51EK43M4QCL22UAB1IYKFJVAGB0I4DQ4DKTNDVH1
CLIENT_SECRET:O5YLENJUJPGDQQYNSBB2MU3MSFATXSR2A3YEUCSNAPLGIMDW


In [32]:
radius = 2000
LIMIT = 100

venues = []

for lat, lon, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhoods']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        lon,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            lon, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [33]:
# convert the venues list into a new DataFrame
address = pd.DataFrame(venues)

# define the column names
address.columns = ['Neighborhood', 'Latitude', 'Longitude', 'AddressName', 'AddressLatitude', 'AddressLongitude', 'AddressCategory']

In [34]:
print(address.shape)
address.head()

(7418, 7)


,Neighborhood,Latitude,Longitude,AddressName,AddressLatitude,AddressLongitude,AddressCategory
0,Alto de Pinheiros,-23.55273,-46.70916,Droga Raia,-23.554750,-46.709082,Pharmacy
1,Alto de Pinheiros,-23.55273,-46.70916,Praça Pôr do Sol,-23.553541,-46.702875,Plaza
2,Alto de Pinheiros,-23.55273,-46.70916,Pão de Açúcar ( Antigo Sé),-23.552717,-46.708607,Market
3,Alto de Pinheiros,-23.55273,-46.70916,Senzala Bar & Grill,-23.554694,-46.708755,Bar
4,Alto de Pinheiros,-23.55273,-46.70916,Praça Conde de Barcelos,-23.552146,-46.713696,Dog Run


In [35]:
address.groupby(["Neighborhood"]).count()

,Latitude,Longitude,AddressName,AddressLatitude,AddressLongitude,AddressCategory
Neighborhood,,,,,,
Alto de Pinheiros,100,100,100,100,100,100
Anhanguera,22,22,22,22,22,22
Aricanduva,100,100,100,100,100,100
Artur Alvim,100,100,100,100,100,100
Barra Funda,100,100,100,100,100,100
Bela Vista,1,1,1,1,1,1
Belem,100,100,100,100,100,100
Bom Retiro,100,100,100,100,100,100
Bras,100,100,100,100,100,100


In [36]:
print('There are {} uniques categories.'.format(len(address['AddressCategory'].unique())))

There are 344 uniques categories.


In [38]:
address['AddressCategory'].unique()[:50]

array(['Pharmacy', 'Plaza', 'Market', 'Bar', 'Dog Run',
       'Other Great Outdoors', 'Restaurant', 'Clothing Store', 'Trail',
       'Recreation Center', 'Playground', 'Bistro', 'Athletics & Sports',
       'Bike Shop', 'Massage Studio', 'College Gym', 'Bookstore',
       'Brazilian Restaurant', 'Beer Bar', "Women's Store",
       'Chocolate Shop', 'Mediterranean Restaurant', 'Supermarket',
       'Pastry Shop', 'Pizza Place', 'Cupcake Shop',
       'Middle Eastern Restaurant', 'Park', 'Spa', 'Theater',
       'Food & Drink Shop', 'Flower Shop', 'Gym / Fitness Center',
       'Breakfast Spot', 'Cosmetics Shop', 'Office', 'Italian Restaurant',
       'Ice Cream Shop', 'Health Food Store', 'Steakhouse',
       'Salon / Barbershop', 'Cocktail Bar', 'Coffee Shop',
       'Burger Joint', 'Shopping Mall', 'Dessert Shop',
       'Vegetarian / Vegan Restaurant', 'Bike Rental / Bike Share',
       'Tennis Court', 'Furniture / Home Store'], dtype=object)

In [40]:
"Neighborhood" in address['AddressCategory'].unique()

True

In [41]:
justone = pd.get_dummies(address[['AddressCategory']], prefix="", prefix_sep="")


justone['Neighborhoods'] = address['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [justone.columns[-1]] + list(justone.columns[:-1])
justone = justone[fixed_columns]

In [42]:
print(justone.shape)
justone.head()

(7418, 345)


,Neighborhoods,Acai House,Accessories Store,Airport,American Restaurant,Amphitheater,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Art Studio,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Baiano Restaurant,Bakery,Bar,Baseball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Bookstore,Borek Place,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Buffet,Building,Burger Joint,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Candy Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,College Quad,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Dive Bar,Dog Run,Donburi Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Electronics Store,Empada House,Empanada Restaurant,Entertainment Service,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Lebanese Restaurant,Library,Lighting Store,Lingerie Store,Liquor Store,Lottery Retailer,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mineiro Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,Nightclub,Northeastern Brazilian Restaurant,Northern Brazilian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoors & Recreation,Outlet Store,Paintball Field,Paper / Office Supplies Store,Park,Pastelaria,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Lab,Pie Shop,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Pub,Public Art,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Rock Club,Roller Rink,Rugby Pitch,Sake Bar,Salad Place,Salon / Barbershop,Samba School,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,Soup Place,South American Restaurant,Southea

In [43]:
spgroup = justone.groupby(["Neighborhoods"]).mean().reset_index()

In [44]:
print(spgroup.shape)
spgroup

(90, 345)


,Neighborhoods,Acai House,Accessories Store,Airport,American Restaurant,Amphitheater,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Art Studio,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Baiano Restaurant,Bakery,Bar,Baseball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Bookstore,Borek Place,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Buffet,Building,Burger Joint,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Candy Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,College Quad,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Dive Bar,Dog Run,Donburi Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Electronics Store,Empada House,Empanada Restaurant,Entertainment Service,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Lebanese Restaurant,Library,Lighting Store,Lingerie Store,Liquor Store,Lottery Retailer,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mineiro Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,Nightclub,Northeastern Brazilian Restaurant,Northern Brazilian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoors & Recreation,Outlet Store,Paintball Field,Paper / Office Supplies Store,Park,Pastelaria,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Lab,Pie Shop,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Pub,Public Art,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Rock Club,Roller Rink,Rugby Pitch,Sake Bar,Salad Place,Salon / Barbershop,Samba School,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,Soup Place,South American Restaurant,Southea

In [45]:
len(spgroup[spgroup["Hotel"] > 0])

22

In [46]:
hotels = spgroup[["Neighborhoods","Hotel"]]

In [47]:
hotels

,Neighborhoods,Hotel
0,Alto de Pinheiros,0.000000
1,Anhanguera,0.000000
2,Aricanduva,0.000000
3,Artur Alvim,0.000000
4,Barra Funda,0.010000
5,Bela Vista,0.000000
6,Belem,0.010000
7,Bom Retiro,0.000000
8,Bras,0.000000
9,Brasilandia,0.000000


### Clustering Neighborhoods

In [50]:
clusters = 7

clustering = hotels.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=clusters, random_state=0).fit(clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 4, 0, 4, 0, 0, 0], dtype=int32)

In [52]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
spmerged = hotels.copy()

# add clustering labels
spmerged["Cluster Labels"] = kmeans.labels_

In [53]:
spmerged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
spmerged.head()

,Neighborhood,Hotel,Cluster Labels
0,Alto de Pinheiros,0.00,0
1,Anhanguera,0.00,0
2,Aricanduva,0.00,0
3,Artur Alvim,0.00,0
4,Barra Funda,0.01,4


In [55]:
spmerged = spmerged.join(neighborhoods.set_index("Neighborhoods"), on="Neighborhood")

print(spmerged.shape)
spmerged.tail() # check the last columns!

(93, 5)


,Neighborhood,Hotel,Cluster Labels,Latitude,Longitude
85,Vila Mariana,0.01,4,-23.58453,-46.63811
86,Vila Matilde,0.00,0,-23.53783,-46.52614
87,Vila Medeiros,0.00,0,-23.48904,-46.58328
88,Vila Prudente,0.00,0,-23.57971,-46.58131
89,Vila Sonia,0.00,0,-23.59457,-46.73133


In [56]:
# sort the results by Cluster Labels
print(spmerged.shape)
spmerged.sort_values(["Cluster Labels"], inplace=True)
spmerged

(93, 5)


,Neighborhood,Hotel,Cluster Labels,Latitude,Longitude
0,Alto de Pinheiros,0.000000,0,-23.552730,-46.709160
59,Ponte Rasa,0.000000,0,-23.513240,-46.504600
58,Pirituba,0.000000,0,-23.466240,-46.723490
57,Pinheiros,0.000000,0,-23.562000,-46.685970
57,Pinheiros,0.000000,0,-23.562000,-46.685970
56,Perus,0.000000,0,-23.402670,-46.749820
54,Penha,0.000000,0,-23.522170,-46.530680
54,Penha,0.000000,0,-23.522170,-46.530680
52,Parque do Carmo,0.000000,0,-23.560600,-46.460350
51,Parelheiros,0.000000,0,-23.821560,-46.702930


In [57]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(clusters)
ys = [i+x+(i*x)**2 for i in range(clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(spmerged['Latitude'], spmerged['Longitude'], spmerged['Neighborhood'], spmerged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [58]:
map_clusters.save('map_clusters.html')

### Examine Clusters

In [59]:
spmerged.loc[spmerged['Cluster Labels'] == 0]

,Neighborhood,Hotel,Cluster Labels,Latitude,Longitude
0,Alto de Pinheiros,0.0,0,-23.552730,-46.709160
59,Ponte Rasa,0.0,0,-23.513240,-46.504600
58,Pirituba,0.0,0,-23.466240,-46.723490
57,Pinheiros,0.0,0,-23.562000,-46.685970
57,Pinheiros,0.0,0,-23.562000,-46.685970
56,Perus,0.0,0,-23.402670,-46.749820
54,Penha,0.0,0,-23.522170,-46.530680
54,Penha,0.0,0,-23.522170,-46.530680
52,Parque do Carmo,0.0,0,-23.560600,-46.460350
51,Parelheiros,0.0,0,-23.821560,-46.702930


In [60]:
spmerged.loc[spmerged['Cluster Labels'] == 1]

,Neighborhood,Hotel,Cluster Labels,Latitude,Longitude
11,Cachoeirinha,0.5,1,-22.73333,-46.58333


In [61]:
spmerged.loc[spmerged['Cluster Labels'] == 2]

,Neighborhood,Hotel,Cluster Labels,Latitude,Longitude
36,Jardim Helena,0.056338,2,-22.72987,-47.29615
74,Socorro,0.053571,2,-22.59307,-46.52874


In [62]:
spmerged.loc[spmerged['Cluster Labels'] == 4]

,Neighborhood,Hotel,Cluster Labels,Latitude,Longitude
6,Belem,0.01,4,-23.53805,-46.59592
85,Vila Mariana,0.01,4,-23.58453,-46.63811
78,Vila Andrade,0.01,4,-23.63327,-46.73701
65,Santo Amaro,0.01,4,-23.65310,-46.70631
64,Santana,0.01,4,-23.49644,-46.62926
81,Vila Guilherme,0.01,4,-23.51489,-46.60770
4,Barra Funda,0.01,4,-23.52910,-46.66208
50,Morumbi,0.01,4,-23.60100,-46.71551
65,Santo Amaro,0.01,4,-23.65310,-46.70631
75,Tatuape,0.01,4,-23.53603,-46.57544


In [63]:
spmerged.loc[spmerged['Cluster Labels'] == 5]

,Neighborhood,Hotel,Cluster Labels,Latitude,Longitude
63,Santa Cecilia,0.020000,5,-23.53592,-46.65775
55,Perdizes,0.020000,5,-23.54057,-46.67236
53,Pedreira,0.019608,5,-22.74142,-46.90986
47,Moema,0.020000,5,-23.60605,-46.66367
43,Limao,0.020000,5,-23.50884,-46.67158
19,Casa Verde,0.020000,5,-23.51152,-46.65629


### Conclusion

Most of the Hotels are concetrated in specific neghborhoods. Neighborhoods in cluster 5 are not a good location for a new Hotel, since it has a high concentration of Hotels and then a high competition. However, we have many neighborhoods with no Hotels and others with only 1 hotels nearby. So I would recommend the stakeholders to consider neighborhoods on cluster 0 and 1 to build a new Hotel.